In [1]:
!pip install tensorflow==1.6

     |████████████████████████████████| 45.9MB 69kB/s 
     |████████████████████████████████| 3.1MB 47.8MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=ff24caf37fd2492baebaa5d094b0774ca4cc6d77d8dfdf13c06d5903125ee8fd
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.4
    Uninstalling bleach-3.1.4:
      Successfully uninstalled bleach-3.1.4
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc3


# **Lyrics Generation wit LSTM**

In [0]:
import tensorflow as tf
import pickle
#print(tensorflow.__version__)
from tensorflow.contrib import rnn

# **Preprocessing Step**

In [5]:
!pip install codecs
import numpy as np
import codecs

ERROR: Could not find a version that satisfies the requirement codecs (from versions: none)
ERROR: No matching distribution found for codecs


In [0]:
class Preprocessing:
  vocabulary = {}
  binary_vocabulary = {}
  char_lookup = {}
  size = 0
  separator = '->'
  def generate(self,input_file_path):
    input_file = codecs.open(input_file_path,'r','utf_8')
    index = 0
    for line in input_file:
      for char in line:
        if char not in self.vocabulary:
          self.vocabulary[char] = index
          self.char_lookup[index] = char
          index +=1
    input_file.close()
    self.set_vocabulary_size()
    self.create_binary_representation()
  def retrieve(self,input_file_path):
    input_file = codecs.open(input_file_path,'r','utf_8')
    buffer = ""
    for line in input_file:
      try:
        separator_position = len(buffer) + line.index(self.separator)
        buffer += line
        key = buffer[:separator_position]
        value = buffer[separator_position + len(self.separator):]
        value = np.fromstring(value,sep=',')
        self.binary_vocabulary[key] = value
        self.vocabulary[key] = np.where(value==1)[0][0]
        self.char_lookup[np.where(value==1)[0][0]] = key
        buffer = ""
      except ValueError:
        buffer +=line
    input_file.close()
    self.set_vocabulary_size()
  def create_binary_representation(self):
    for key,value in self.vocabulary.iteritems():
      binary = np.zeros(self.size)
      binary[value] = 1
      self.binary_vocabulary[key] = binary
  def set_vocabulary_size(self):
    self.size = len(self.vocabulary)
    print("Vocabulary Size: {}".format(self.size))
  def get_serialized_binary_representation(self):
    string = ""
    np.set_printoptions(threshold='nan')
    for key,value in self.binary_vocabulary.iteritems():
      array_as_string  = np.array2string(value,separator= ',',max_line_width = self.size * self.size)
      string  +="{}{}{}\n".format(key.encode('utf-8'),self.separator,array_as_string[1:len(array_as_string) -1])
    return string


# **Defining Model**

In [0]:
class Model:
  x = None
  y = None
  sequence_length = None
  weights = None
  baises = None
  outputs = None

  def __init__(self,name):
    self.name = name
  
  def build(self,input_number, sequence_length,layers_number,units_number,output_number):
    self.x = tf.placeholder("float",[None,sequence_length,input_number])
    self.y = tf.placeholder("float",[None,output_number])
    self.sequence_length =sequence_length

    self.weights = {
        'out':tf.Variable(tf.random_normal([units_number,output_number]))
    }
    self.baises = {
        'out': tf.Variable(tf.random_normal([output_number]))
    }
    x = tf.transpose(self.x,[1,0,2])
    x = tf.reshape(x,[-1,input_number])
    x = tf.split(x,sequence_length,0)
    lstm_layers = []
    for i in range(0,layers_number):
      lstm_layer = rnn.BasicLSTMCell(units_number)
      lstm_layer.append(lstm_layer)
    deep_lstm = rnn.MultiRNNCell(lstm_layers)
    self.outputs, states = rnn.static_rnn(deep_lstm,x,dtype = tf.float32)
    print("Build model with input number:{},sequence_length:{},layers_number:{},"\
    " units_number:{},output_number:{}".format(input_number,sequence_length,layers_number,units_number,output_number))
    self.save(input_number,sequence_length,layers_number,units_number,output_number)  
  def save(self,input_number, sequence_length,layers_number,units_number,output_number):
    config = {
        "input_number":input_number,
        "sequence_length":sequence_length,
        "layers_number":layers_number,
        "units_number":units_number,
        "output_number":output_number
    }
    config_file = open(self.get_config_file_path(),'w')
    pickle.dump(config,config_file)
    config_file.close()
  def restore(self):
    config_file = open(self.get_config_file_path(),'r')
    config = pickle.load(config_file)
    config_file.close()

    self.build(config["input number"],config["sequence length"],config["layers_number"],
               config["units_number"],config["output_number"])
  def get_classifier(self):
    return tf.matmul(self.outputs[-1], self.weights['out']) + self.baises['out']
  def get_config_file_path(self):
    return "{}/{}.config".format(self.name,self.name)

# **Train Deep Tensorflow based LSTM Model**

In [0]:
import argparse
import codecs
from collections import deque

In [18]:

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--training_file', type=str, required=True)
    parser.add_argument('--vocabulary_file', type=str, required=True)
    parser.add_argument('--model_name', type=str, required=True)

    parser.add_argument('--epoch', type=int, default=200)
    parser.add_argument('--batch_size', type=int, default=50)
    parser.add_argument('--sequence_length', type=int, default=50)
    parser.add_argument('--log_frequency', type=int, default=100)
    parser.add_argument('--learning_rate', type=int, default=0.002)
    parser.add_argument('--units_number', type=int, default=128)
    parser.add_argument('--layers_number', type=int, default=2)
    args = parser.parse_args()

    training_file = args.training_file
    vocabulary_file = args.vocabulary_file
    model_name = args.model_name

    epoch = args.epoch
    batch_size = args.batch_size
    sequence_length = args.sequence_length
    log_frequency = args.log_frequency
    learning_rate = args.learning_rate

    batch = Batch(training_file, vocabulary_file, batch_size, sequence_length)

    input_number = batch.vocabulary.size
    classes_number = batch.vocabulary.size
    units_number = args.units_number
    layers_number = args.layers_number

    print("Start training with epoch: {}, batch_size: {}, log_frequency: {}," \
          "learning_rate: {}".format(epoch, batch_size, log_frequency, learning_rate))

    if not os.path.exists(model_name):
        os.makedirs(model_name)

    model = Model(model_name)
    model.build(input_number, sequence_length, layers_number, units_number, classes_number)
    classifier = model.get_classifier()

    cost = tf.reduce_mean(tf.square(classifier - model.y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    expected_prediction = tf.equal(tf.argmax(classifier, 1), tf.argmax(model.y, 1))
    accuracy = tf.reduce_mean(tf.cast(expected_prediction, tf.float32))
    
    loss_summary = tf.summary.scalar("loss", cost)
    acc_summary = tf.summary.scalar("accuracy", accuracy)
    
    train_summary_op = tf.summary.merge_all()
    out_dir = "{}/{}".format(model_name, model_name)
    train_summary_dir = os.path.join(out_dir, "summaries")
    

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        iteration = 0
        train_summary_writer = tf.summary.FileWriter(train_summary_dir,sess.graph)
        
        while batch.dataset_full_passes < epoch:
            iteration += 1
            batch_x, batch_y = batch.get_next_batch()
            batch_x = batch_x.reshape((batch_size, sequence_length, input_number))

            sess.run(optimizer, feed_dict={model.x: batch_x, model.y: batch_y})
            if iteration % log_frequency == 0:
                acc,summaries = sess.run([accuracy,train_summary_op], feed_dict={model.x: batch_x, model.y: batch_y})
                loss = sess.run(cost, feed_dict={model.x: batch_x, model.y: batch_y})
                print("Iteration {}, batch loss: {:.6f}, training accuracy: {:.5f}".format(iteration * batch_size,
                                                                                           loss, acc))
                train_summary_writer.add_summary(summaries, iteration * batch_size)
        batch.clean()
        print("Optimization done")

        saver = tf.train.Saver(tf.global_variables())
        checkpoint_path = "{}/{}.ckpt".format(model_name, model_name)
        saver.save(sess, checkpoint_path, global_step=iteration * batch_size)
        print("Model saved in {}".format(model_name))

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] --training_file TRAINING_FILE
                             --vocabulary_file VOCABULARY_FILE --model_name
                             MODEL_NAME [--epoch EPOCH]
                             [--batch_size BATCH_SIZE]
                             [--sequence_length SEQUENCE_LENGTH]
                             [--log_frequency LOG_FREQUENCY]
                             [--learning_rate LEARNING_RATE]
                             [--units_number UNITS_NUMBER]
                             [--layers_number LAYERS_NUMBER]
ipykernel_launcher.py: error: the following arguments are required: --training_file, --vocabulary_file, --model_name


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name', type=str, required=True)
    parser.add_argument('--vocabulary_file', type=str, required=True)
    parser.add_argument('--output_file', type=str, required=True)

    parser.add_argument('--seed', type=str, default="Yeah, oho ")
    parser.add_argument('--sample_length', type=int, default=1500)
    parser.add_argument('--log_frequency', type=int, default=100)
    args = parser.parse_args()

    model_name = args.model_name
    vocabulary_file = args.vocabulary_file
    output_file = args.output_file
    seed = args.seed.decode('utf-8')
    sample_length = args.sample_length
    log_frequency = args.log_frequency

    model = Model(model_name)
    model.restore()
    classifier = model.get_classifier()

    vocabulary = Preprocessing()
    vocabulary.retrieve(vocabulary_file)

    sample_file = codecs.open(output_file, 'w', 'utf_8')

    stack = deque([])
    for i in range(0, model.sequence_length - len(seed)):
        stack.append(u' ')

    for char in seed:
        if char not in vocabulary.vocabulary:
            print(char,"is not in vocabulary file")
            char = u' '
        stack.append(char)
        sample_file.write(char)

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(model_name)

        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)

            for i in range(0, sample_length):
                vector = []
                for char in stack:
                    vector.append(vocabulary.binary_vocabulary[char])
                vector = np.array([vector])
                prediction = sess.run(classifier, feed_dict={model.x: vector})
                predicted_char = vocabulary.char_lookup[np.argmax(prediction)]

                stack.popleft()
                stack.append(predicted_char)
                sample_file.write(predicted_char)

                if i % log_frequency == 0:
                    print("Progress: {}%".format((i * 100) / sample_length))

            sample_file.close()
            print("Lyrics saved in {}".format(output_file))

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] --model_name MODEL_NAME --vocabulary_file
                             VOCABULARY_FILE --output_file OUTPUT_FILE
                             [--seed SEED] [--sample_length SAMPLE_LENGTH]
                             [--log_frequency LOG_FREQUENCY]
ipykernel_launcher.py: error: the following arguments are required: --model_name, --vocabulary_file, --output_file


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
